# Getting data from the SSC RESTful Web Services API

In [1]:
# Import dependencies
import requests 
import sscws
from datetime import datetime, timedelta
import math
import pandas as pd
from sscws.sscws import SscWs


## Define helper functions to interpret the data from the API

In [2]:
# Get last data point and subtract 2 hours
def subtract_two_hours(iso_string):
    # Convert the ISO string to a datetime object
    dt = datetime.fromisoformat(iso_string)
    
    # Subtract one hour from the datetime object
    new_dt = dt - timedelta(hours=2)
    
    # Convert back to ISO format
    return new_dt.isoformat()

In [3]:
def distance_from_earth(x, y, z):
    """
    Calculate the distance from the Earth's center to a point in space given its X, Y, and Z coordinates.

    Parameters:
    x (float): X coordinate in the GSE coordinate system (in kilometers)
    y (float): Y coordinate in the GSE coordinate system (in kilometers)
    z (float): Z coordinate in the GSE coordinate system (in kilometers)

    Returns:
    float: Distance from the Earth's center to the point (in kilometers)
    """
    distance = math.sqrt(x**2 + y**2 + z**2)
    return distance


In [4]:
def convert_to_earth_coordinates(gse_longitude, gse_latitude):
    """
    Convert GSE coordinate system longitude and latitude into conventional Earth mapping longitude and latitude.

    GSE Longitude is given in the range [0, 360]. It is converted to the range [-180, 180] for Earth mapping.
    GSE Latitude is the same as Earth's latitude and ranges from [-90, 90].

    Parameters:
    gse_longitude (float): Longitude in the GSE coordinate system (0 to 360 degrees)
    gse_latitude (float): Latitude in the GSE coordinate system (-90 to 90 degrees)

    Returns:
    tuple: Converted longitude and latitude for Earth mapping
    """
    # Convert GSE longitude to Earth mapping longitude
    if gse_longitude > 180:
        earth_longitude = gse_longitude - 360
    else:
        earth_longitude = gse_longitude

    # GSE latitude is the same as Earth's latitude
    earth_latitude = gse_latitude

    return earth_longitude, earth_latitude


In [5]:
def distance_from_earth_surface(x, y, z):
    """
    Calculate the distance from the Earth's surface to a point in space given its X, Y, and Z coordinates
    in the Geocentric Solar Ecliptic (GSE) coordinate system. The coordinates are assumed to be in kilometers (km).
    The Earth's average radius (about 6,371 km) is subtracted from the calculated distance.

    Parameters:
    x (float): X coordinate in the GSE coordinate system (in kilometers)
    y (float): Y coordinate in the GSE coordinate system (in kilometers)
    z (float): Z coordinate in the GSE coordinate system (in kilometers)

    Returns:
    float: Distance from the Earth's surface to the point (in kilometers)
    """
    earth_radius_km = 6371  # Earth's average radius in kilometers
    distance_from_center = math.sqrt(x**2 + y**2 + z**2)
    distance_from_surface = max(distance_from_center - earth_radius_km, 0) # Ensure non-negative distance
    return distance_from_surface

## Get the data from the API

In [6]:
# Get the initial dataset from the API
ssc = SscWs()
satellites_result = ssc.get_observatories()

In [7]:
# Parse through results and get the GSE coordinate data
coords_dict = {}

for satellite in satellites_result['Observatory']:
    
    # Set up time window for data (get latest data)
    end_time_iso = satellite['EndTime'].isoformat()
    end_time_minus_one_hour = subtract_two_hours(end_time_iso)

    # Get the locations for each satellite
    locations = ssc.get_locations([satellite['Id']], [end_time_minus_one_hour, end_time_iso])
    
    # Get the data where the coordinates are
    data = locations['Data'][0]
    coords = data['Coordinates'][0]
    
    # Store the results for each satellite
    coords_dict[satellite['Name']] = coords

In [8]:
# Convert GSE data into regular lat, long, and distance and store in dataframe
sat_locations_df = pd.DataFrame()

for key, value in coords_dict.items():
    sat_name = key
    sat_long, sat_lat = convert_to_earth_coordinates(value['Longitude'][-1], value['Latitude'][-1])
    sat_radius = distance_from_earth(value['X'][-1], value['Y'][-1], value['Z'][-1])
    sat_distance_from_surface = distance_from_earth_surface(value['X'][-1], value['Y'][-1], value['Z'][-1])
    
    # Create a DataFrame for the current satellite's details
    sat_details_df = pd.DataFrame([{
        'Satellite Name': sat_name,
        'Latitude': sat_lat,
        'Longitude': sat_long,
        'Distance from Earth Center (km)': sat_radius,
        'Distance from Earth Surface (km)': sat_distance_from_surface
    }])
    
    # Concatenate the new DataFrame with the existing DataFrame
    sat_locations_df = pd.concat([sat_locations_df, sat_details_df], ignore_index=True)
    
    

In [9]:
# Filter data for objects that are not farther than the moon
sat_locations_df = sat_locations_df[sat_locations_df['Distance from Earth Center (km)'] > 6371]
sat_locations_df = sat_locations_df[sat_locations_df['Distance from Earth Surface (km)'] < 360000]
sat_locations_df

,Satellite Name,Latitude,Longitude,Distance from Earth Center (km),Distance from Earth Surface (km)
1,Active,-34.928585,82.613740,7104.831958,733.831958
2,AE-C,-11.169291,72.150070,6568.678092,197.678092
3,AE-D,-5.791222,-124.893890,8369.291638,1998.291638
4,AE-E,-15.266993,110.649994,6543.071954,172.071954
5,AEROCUBE-6A,-43.780540,-63.022900,7048.082644,677.082644
...,...,...,...,...,...
266,Twins2,9.484488,-20.947850,13823.892991,7452.892991
267,UARS,-38.690254,170.185780,6887.613536,516.613536
268,Viking,-33.478497,-68.997470,18187.559456,11816.559456
270,XMM-Newton,9.431140,170.833160,39395.672421,33024.672421


In [11]:
# Save to csv
sat_locations_df.to_csv('sat_locations.csv', index=False)
